# First Year Project - Project 1 - Corona and Weather

## Here importing the libraries required to run all the code
* [Pandas Documentation](https://pandas.pydata.org/docs/)
* [Numpy Documentation](https://numpy.org/doc/)
* [Folium Documentation](https://python-visualization.github.io/folium/)
* [Json Documentation](https://docs.python.org/3/library/json.html)
* [Statsmodels Documentation](https://www.statsmodels.org/stable/)
* [Scipy Documentation](https://scipy.github.io/devdocs/index.html)
* [Seaborn Documentation](https://seaborn.pydata.org/)

In [39]:
#Importing needed libraries
import pandas as pd
import numpy as np
import folium, json
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
from statsmodels.stats.multitest import multipletests
import seaborn as sns

### Task 0: Data filtering and cleaning

In [48]:
# Read comma-separated values (csv) file into pandas DataFrame
data = pd.read_csv("../../Data/raw/weather/weather.csv")

# A processed data set that contains only relevant fields and records 
is_DE = data[data['iso3166-2'].str.startswith('DE')]


is_DE.describe()

,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
count,6000.000000,6.000000e+03,6.000000e+03,6000.000000,6000.000000,6000.000000,6000.000000
mean,74.332120,6.678336e+06,2.379588e+06,283.281258,0.002206,16.050800,3.503221
std,13.595421,6.212940e+06,5.121144e+04,7.146610,0.003439,14.515517,1.438837
min,33.880265,0.000000e+00,2.212828e+06,260.531714,0.000000,0.000000,1.091346
25%,64.268213,1.090176e+06,2.343795e+06,277.837390,0.000054,2.500231,2.427226
50%,76.469758,4.610296e+06,2.385826e+06,283.228645,0.000684,12.720154,3.202055
75%,85.703146,1.150929e+07,2.422283e+06,288.872994,0.002907,27.398392,4.261427
max,98.264247,2.370823e+07,2.497243e+06,300.960922,0.031971,52.792235,11.221876


In [52]:
df = pd.read_csv("../../Data/processed/Weather+Cases_DE.csv")
corona_df = pd.read_csv("../../Data/processed/de_corona.csv")
# case_by_region
# case_by_region = case_by_region.set_index(['iso3166-2', case_by_region.index])
# case_by_region
df


,Unnamed: 0,date,region_code,confirmed_addition,deceased_addition,iso3166-2,population,cases_pc,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
0,0,2020-02-16,Mecklenburg-Vorpommern,1,0,DE-MV,1609675,6.212434e-07,69.818665,4.562424e+04,2.392858e+06,11.144049,0.004641,0.211216,7.888859
1,1,2020-02-17,Brandenburg,1,0,DE-BB,2511917,3.981023e-07,64.189273,9.007504e+05,2.409194e+06,9.162595,0.001005,2.631685,6.319781
2,2,2020-02-18,Brandenburg,1,0,DE-BB,2511917,3.981023e-07,71.904630,4.042940e+06,2.421421e+06,6.529468,0.002388,4.037911,6.150824
3,3,2020-02-20,Nordrhein-Westfalen,1,0,DE-NW,17932651,5.576420e-08,81.738535,1.064802e+06,2.383996e+06,6.513867,0.006802,3.524003,6.477330
4,4,2020-02-24,Baden-Württemberg,1,0,DE-BW,11069533,9.033805e-08,80.314208,6.604918e+05,2.308155e+06,8.744530,0.005284,5.613589,4.663207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5575,5575,2021-02-21,Rheinland-Pfalz,120,0,DE-RP,4084844,2.937689e-05,65.642820,8.457394e+06,2.348996e+06,8.973657,0.000000,7.879810,3.009041
5576,5576,2021-02-21,Sachsen,68,0,DE-SN,4077937,1.667510e-05,74.638170,6.687276e+06,2.363996e+06,6.127820,0.000008,5.409593,4.648033
5577,5577,2021-02-21,Sachsen-Anhalt,84,0,DE-ST,2208321,3.803795e-05,69.587170,7.276057e+06,2.408002e+06,9.268775,0.000000,5.273630,2.972845
5578,5578,2021-02-21,Schleswig-Holstein,113,0,DE-SH,2896712,3.900975e-05,78.077610,5.698494e+06,2.431487e+06,8.793486,0.000000,4.474084,2.532838
